In [2]:
import pymongo
import json

In [3]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["twitter"]

In [4]:
#db.tweets.drop()

In [6]:
selected_columns = ['id', 'created_at', 'text', 'source', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities']

def filter_func(item):
    key, value = item
    
    if key in selected_columns:
        return True
    else:
        return False

In [7]:
def cleaning(data):
        
        doc = dict(filter(filter_func, data.items()))
        doc['user_id'] = data['user']['id']      
        doc['_id'] = doc['id']
        
        #formating the date field
        format =  "%a %b %d %H:%M:%S %z %Y"
        obj = strptime(doc['created_at'], format)
        doc['created_at'] = strftime('%Y-%m-%d %H:%M:%S', obj)
        
        #extracting hashtags
        hashtag_list = []
        for i in doc['entities']['hashtags']:
                hashtag_list.append(i['text'])
        doc['hashtags'] = hashtag_list  
        
        #Extracting user mentions
        user_mentions = []
        for i in doc['entities']['user_mentions']:
            user_mentions.append(i['screen_name'])
        doc['user_mentions'] = user_mentions
        
        del doc['id']
        del doc['entities']
        return doc

In [8]:
def load_data(file_name):
    with open(file_name, "r") as f1:
        for line in f1:
            try:
                data = json.loads(line)
                doc = cleaning(data)

                if 'retweeted_status' in data:
                    doc['retweeted_status_id'] = data['retweeted_status']['id']
                    retweet = cleaning(data['retweeted_status'])

                    try:
                        db.tweets.insert_one(retweet)
                    except Exception:
                        pass

                db.tweets.insert_one(doc)

            except Exception:
                continue

In [9]:
load_data("corona-out-3")

In [7]:
db.tweets.count_documents({})

112023